# A/B Testing Course

## Lesson 1. Basic Statistics

### Homework

#### Import Libraries

In [1]:
import os
import pandas as pd

#### Import Data

In [2]:
URL_BASE = ''

def read_database(file_name):
    return pd.read_csv(os.path.join(URL_BASE, file_name))

In [3]:
df_sales = read_database('2022-04-01T12_df_sales.csv')
df_web_logs = read_database('2022-04-01T12_df_web_logs.csv')

df_sales['date'] = pd.to_datetime(df_sales['date'])
df_web_logs['date'] = pd.to_datetime(df_web_logs['date'])

#### Data Description

df_sales - information about purchases, one row represents one order:
- sale_id - purchase identifier;
- date - date of purchase;
- count_pizza - number of pizzas in the order;
- count_drink - number of drinks in the order;
- price - order price;
- user_id - user identifier.   

df_web_logs - a table with logs of website visits:
- user_id - user identifier;
- page - which page of the website was opened. Page types:
    - m - menu page;
    - b - cart and order confirmation page;
    - p - payment page;
- date - date of the page opening;
- load_time - page load time in milliseconds.

#### Checking Data

In [6]:
df_sales.head()

,sale_id,date,count_pizza,count_drink,price,user_id
0,1000001,2022-02-04 10:00:24,1,0,720,1c1543
1,1000002,2022-02-04 10:02:28,1,1,930,a9a6e8
2,1000003,2022-02-04 10:02:35,3,1,1980,23420a
3,1000004,2022-02-04 10:03:06,1,1,750,3e8ed5
4,1000005,2022-02-04 10:03:23,1,1,870,cbc468


In [8]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203847 entries, 0 to 203846
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   sale_id      203847 non-null  int64         
 1   date         203847 non-null  datetime64[ns]
 2   count_pizza  203847 non-null  int64         
 3   count_drink  203847 non-null  int64         
 4   price        203847 non-null  int64         
 5   user_id      203847 non-null  object        
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 9.3+ MB


In [7]:
df_web_logs.head()

,user_id,page,date,load_time
0,f25239,m,2022-02-03 23:45:37,80.8
1,06d6df,m,2022-02-03 23:49:56,70.5
2,06d6df,m,2022-02-03 23:51:16,89.7
3,f25239,m,2022-02-03 23:51:43,74.4
4,697870,m,2022-02-03 23:53:12,66.8


In [9]:
df_web_logs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1964507 entries, 0 to 1964506
Data columns (total 4 columns):
 #   Column     Dtype         
---  ------     -----         
 0   user_id    object        
 1   page       object        
 2   date       datetime64[ns]
 3   load_time  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 60.0+ MB


#### Task 1. 

Determine the top 3 products by total revenue.

In [ ]:
tbc..